#### Student Name:
#### Student ID:

# CSE 190 Project




In [1]:
import numpy as np
import random
import matplotlib.pyplot as plt
%matplotlib inline
from scipy.io import wavfile
from numpy.linalg import svd
from scipy.stats.mstats import gmean
from matplotlib import rcParams
import scipy
import os
import sys
import glob
import pickle
from music21 import *
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import LSTM
from keras.layers import Activation
from keras.layers import BatchNormalization as BatchNorm
from keras.layers import ReLU
from keras.layers import Lambda
from keras.utils import np_utils
from keras.callbacks import ModelCheckpoint
from keras.losses import categorical_crossentropy
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()

Transpose all the midi files into key of C major or A minor

reference: https://gist.github.com/aldous-rey/68c6c43450517aa47474

In [ ]:
# major conversions
majors = dict([("A-", 4),("A", 3),("B-", 2),("B", 1),("C", 0),("D-", -1),("D", -2),("E-", -3),("E", -4),("F", -5),("G-", 6),("G", 5)])
minors = dict([("A-", 1),("A", 0),("B-", -1),("B", -2),("C", -3),("D-", -4),("D", -5),("E-", 6),("E", 5),("F", 4),("G-", 3),("G", 2)])


#os.chdir("./")
for file in glob.glob("mozart_piano_sonata/*.mid"):
    midi = converter.parse(file)
    key = midi.analyze('key')
    file=file[file.find("\\")+1:len(file)-4]
#    print key.tonic.name, key.mode
    if key.mode == "major":
        halfSteps = majors[key.tonic.name]
        
    elif key.mode == "minor":
        halfSteps = minors[key.tonic.name]
    
    midi = midi.transpose(halfSteps)
    midi.write('midi',fp='transpose/{0}_transpose.mid'.format(file))

In [4]:
def create_network(network_input, n_vocab):

    model = Sequential()
    model.add(LSTM(
        512,
        input_shape=(network_input.shape[1], network_input.shape[2]),
        recurrent_dropout=0.3,
        return_sequences=True
    ))
    
    ''' Your Code Here '''
    model.add(LSTM(512,recurrent_dropout=0.3))
    model.add(BatchNorm())
    model.add(Dropout(0.3))
    model.add(Dense(256))
    model.add(Activation('relu'))
    model.add(BatchNorm())
    model.add(Dropout(0.3))
    model.add(Dense(n_vocab))
    model.add(Lambda(lambda x: x / 0.6))
    model.add(Activation('softmax'))
    model.compile(optimizer='adam', loss='categorical_crossentropy')

    
    
    return model

After parsing the midi files into musi21 stream, if the element is a note element, get the pitch name and duration, and encoded in the format {pitchname} _ {note duration}. If the element is a chord element, encode in normal chord form.

In [6]:
def get_notes():

    notes = []
    for file in glob.glob("transpose/*.mid"):
        midi = converter.parse(file)
        print("Parsing %s" % file)
        notes_to_parse = None
        try: # file has instrument parts
            s2 = instrument.partitionByInstrument(midi)
            notes_to_parse = s2.parts[0].recurse() 
        except: # file has notes in a flat structure
            notes_to_parse = midi.flat.notes

        for element in notes_to_parse:
            if isinstance(element, note.Note):
                notes.append(str(element.pitch)+'_'+str(element.duration.quarterLength))
            elif isinstance(element, chord.Chord):
                notes.append('.'.join(str(n) for n in element.normalOrder))
            #print(element)
            #print(notes)
    pickle.dump(notes, open('notes.p', 'wb'))
    #print(notes)
    return notes
def prepare_sequences(notes, n_vocab):
    """ Prepare the sequences used by the Neural Network """
    sequence_length = 4 

    pitchnames = sorted(set(item for item in notes))
    #print(pitchnames)

    note_to_int = dict((note, number) for number, note in enumerate(pitchnames))
    #print(note_to_int)
    network_input = []
    network_output = []
    #print(notes)
    for i in range(0, len(notes) - sequence_length, 1):
        sequence_in = notes[i:i + sequence_length]
        #print('in',i,sequence_in)
        sequence_out = notes[i + sequence_length]
        #print('out',i,sequence_out)
        network_input.append([note_to_int[char] for char in sequence_in])
        network_output.append(note_to_int[sequence_out])
    #print(network_input)
    n_patterns = len(network_input)

    # reshape the input into a format compatible with LSTM layers
    network_input = np.reshape(network_input, (n_patterns, sequence_length, 1))
    # normalize input
    network_input = network_input / float(n_vocab)

    network_output = np_utils.to_categorical(network_output)
    
    #print(network_input)

    return (network_input, network_output)


Train the network 50 times.

In [164]:
def train_network():
    """ Train a Neural Network to generate music """
    notes = get_notes()

    n_vocab = len(set(notes))
    
    network_input, network_output = prepare_sequences(notes, n_vocab)
    
    model = create_network(network_input, n_vocab)
 
    checkpoint = ModelCheckpoint(
        "weights2-improvement-{epoch:02d}-{loss:.4f}-bigger.hdf5",
        monitor='loss',
        verbose=0,
        save_best_only=True,
        mode='min'
    )
    
    callbacks_list = [checkpoint]

    # Your line of code here
    model.fit(x=network_input,y=network_output,callbacks=callbacks_list,epochs=50)

I did the training in another file so here is an error. The weighting I used below is the weighting I got from training in another ipynb file.

In [54]:
train_network()


NameError: name 'train_network' is not defined

In [61]:
def prepare_sequences_prediction(notes, pitchnames, n_vocab):

    note_to_int = dict((note, number) for number, note in enumerate(pitchnames))
    sequence_length = 4
    network_input = []
    output = []
    for i in range(0, len(notes) - sequence_length, 1):
        sequence_in = notes[i:i + sequence_length]
        sequence_out = notes[i + sequence_length]
        network_input.append([note_to_int[char] for char in sequence_in])
        output.append(note_to_int[sequence_out])
        
    print(note_to_int)

    n_patterns = len(network_input)

    # reshape the input into a format compatible with LSTM layers
    normalized_input = np.reshape(network_input, (n_patterns, sequence_length, 1))
    # normalize input
    normalized_input = normalized_input / float(n_vocab)

    return (network_input, normalized_input)

def generate_notes(model, network_input, pitchnames, n_vocab):
    """ Generate notes from the neural network based on a sequence of notes """
    # Starts the melody by picking a random sequence from the input as a starting point
    start = np.random.randint(0, len(network_input)-1)

    int_to_note = dict((number, note) for number, note in enumerate(pitchnames))

    pattern = network_input[start]
    prediction_output = []

    for note_index in range(200):
        prediction_input = np.reshape(pattern, (1, len(pattern), 1))
        prediction_input = prediction_input / float(n_vocab)

        
        ### Complete the line below
        prediction = model.predict(prediction_input)
        #print(prediction)
        index = np.argmax(prediction)
        #print(index)
        result = int_to_note[index]
        prediction_output.append(result)

        pattern.append(index)
        pattern = pattern[1:len(pattern)]

    return prediction_output

In [99]:
def generate():
    notes = pickle.load(open('notes.p', 'rb'))
    pitchnames = sorted(set(item for item in notes))
    n_vocab = len(set(notes))

    network_input, normalized_input = prepare_sequences_prediction(notes, pitchnames, n_vocab)
    model = create_network(normalized_input, n_vocab)
    #model.save_weights('weight')
    ### Add a line to load the weights here
    model.load_weights('weights2-improvement-50-4.3253-bigger.hdf5')
    
    
    prediction_output = generate_notes(model, network_input, pitchnames, n_vocab)
    create_midi(prediction_output)

Create midi files by coverting string back to midi notes. Offset are set as the duration of notes that are predicted.

In [129]:
from fractions import Fraction
def create_midi(prediction_output):
    offset = 0
    output_notes = []
    for pattern in prediction_output:
        if ('_' not in pattern) or pattern.isdigit():
            notes_in_chord = pattern.split('.')
            notes = []
            for current_note in notes_in_chord:
                new_note = note.Note(int(current_note))
                new_note.storedInstrument = instrument.Piano()
                notes.append(new_note)
            new_chord = chord.Chord(notes)
            new_chord.offset = offset
            output_notes.append(new_chord)
            offset += 0.5
        else:
            new_note = note.Note(pattern[0:pattern.find('_')],quarterLength=float(Fraction(pattern[pattern.find('_')+1:])))
            new_note.offset = offset
            #print(offset)
            new_note.storedInstrument = instrument.Piano()
            output_notes.append(new_note)
            offset += float(Fraction(pattern[pattern.find('_')+1:]))
    midi_stream = stream.Stream(output_notes)
    midi_stream.write('midi', fp='test_output.mid')
    
generate()

{'0': 0, '0.1': 1, '0.2': 2, '0.2.5': 3, '0.2.5.7': 4, '0.2.6': 5, '0.2.7': 6, '0.3': 7, '0.3.6': 8, '0.3.6.9': 9, '0.3.7': 10, '0.4': 11, '0.4.5': 12, '0.4.5.7': 13, '0.4.7': 14, '0.5': 15, '0.6': 16, '1': 17, '1.2': 18, '1.2.6': 19, '1.3': 20, '1.4': 21, '1.4.7': 22, '1.4.7.10': 23, '1.4.7.9': 24, '1.5': 25, '1.7': 26, '10': 27, '10.0': 28, '10.0.4': 29, '10.1': 30, '10.1.3': 31, '10.1.4': 32, '10.2': 33, '10.2.4': 34, '10.2.5': 35, '10.3': 36, '11': 37, '11.0': 38, '11.0.2': 39, '11.0.2.4': 40, '11.1': 41, '11.1.5': 42, '11.2': 43, '11.2.4': 44, '11.2.4.7': 45, '11.2.5': 46, '11.2.5.7': 47, '11.3': 48, '11.4': 49, '2': 50, '2.3': 51, '2.4': 52, '2.4.5': 53, '2.4.7': 54, '2.4.8': 55, '2.5': 56, '2.5.7': 57, '2.5.8': 58, '2.5.8.10': 59, '2.5.8.11': 60, '2.5.9': 61, '2.6': 62, '2.6.9': 63, '2.7': 64, '2.8': 65, '3': 66, '3.5': 67, '3.5.9': 68, '3.6': 69, '3.6.9': 70, '3.7': 71, '3.7.10': 72, '3.7.9': 73, '3.8': 74, '3.9': 75, '4': 76, '4.10': 77, '4.5': 78, '4.6': 79, '4.7': 80, '4.7.1

B4_9.5
0.5
C5_3.0
D5_5.25
0.5
A5_0.25
0.5
0.4
B4_0.25
E4_5.25
C5_0.25
B5_0.25
E3_0.25
C5_0.25
G4_0.25
E5_0.25
C5_1.0
G4_0.5
G5_0.25
A5_0.25
G4_0.25
F5_0.25
E5_0.25
D5_1/3
B3_1.0
E5_0.5
D5_0.25
B3_0.25
G3_0.5
B4_0.25
D3_0.25
B3_0.25
G3_0.25
G3_0.25
0.4
G4_0.25
A5_0.25
G5_0.5
11.2
C6_0.25
E5_0.25
G5_0.25
E5_0.25
F5_0.25
D5_0.25
5.9
B4_0.25
G4_0.25
D5_0.25
A3_1.0
G4_0.25
B4_0.25
A4_0.25
G3_0.5
C5_0.5
F#4_1.0
G4_0.25
E4_0.25
B3_0.25
G4_0.25
D5_0.25
C5_0.25
A3_0.5
D5_0.25
B4_0.25
G3_0.25
F4_0.5
G4_0.5
G5_0.25
E4_0.25
E5_0.25
G4_0.25
C6_1.0
E4_0.25
A4_0.25
C5_0.5
F4_0.25
A4_0.25
A4_0.25
C#3_1.0
G4_0.5
E4_0.25
C5_0.25
C4_0.25
E4_0.25
G3_0.25
C4_0.25
C2_0.25
E2_0.25
E3_0.25
E4_0.25
E3_0.25
E-5_0.25
C5_2.0
C4_0.25
E4_0.25
E5_0.25
11.2
C2_0.5
E5_0.25


Generate music by covering top x probabilities in the prediction array, 

In [128]:
import random

def generate_notes(model, network_input, pitchnames, n_vocab):
    """ Generate notes from the neural network based on a sequence of notes """
    # Starts the melody by picking a random sequence from the input as a starting point
    start = np.random.randint(0, len(network_input)-1)

    int_to_note = dict((number, note) for number, note in enumerate(pitchnames))

    pattern = network_input[start]
    prediction_output = []
    prediction_outputA=[]
    prediction_outputB=[]

    #x=0
    for note_index in range(100):
        prediction_input = np.reshape(pattern, (1, len(pattern), 1))
        prediction_input = prediction_input / float(n_vocab)

        
        ### Copy the line below from your above implementation.
        prediction = model.predict(prediction_input)
        top5arg = np.argpartition(-prediction[0], 20)
        top5_args = top5arg[:20]    
        top5 = np.partition(-prediction[0], 20)
        top5_result = -top5[:20]
        index=random.choices(top5_args,top5_result)
        '''index=random.choices(top5_args,top5_result)
        prediction = model.predict(prediction_input)
        
        #print(type(index))
        index = random.choices(np.arange(len(int_to_note)),prediction[0])'''
        print(int_to_note[index[0]])
        result = int_to_note[index[0]]
        prediction_output.append(result)

        pattern.append(index[0])
        pattern = pattern[1:len(pattern)]
        

    
    return prediction_output

Generate two sections of music by using top 5 probabilities in the prediction array and top 15 probabilities in the prediction array respectively. And then parse the two section in ABA form.

In [ ]:
import random

def generate_notes(model, network_input, pitchnames, n_vocab):
    """ Generate notes from the neural network based on a sequence of notes """
    # Starts the melody by picking a random sequence from the input as a starting point
    start = np.random.randint(0, len(network_input)-1)

    int_to_note = dict((number, note) for number, note in enumerate(pitchnames))

    pattern = network_input[start]
    prediction_output = []
    prediction_outputA=[]
    prediction_outputB=[]

    #x=0
    for note_index in range(50):
        prediction_input = np.reshape(pattern, (1, len(pattern), 1))
        prediction_input = prediction_input / float(n_vocab)

        
        ### Copy the line below from your above implementation.
        prediction = model.predict(prediction_input)
        
        top5arg = np.argpartition(-prediction[0], 5)
        top5_args = top5arg[:5]
        #print(top5_args)
        
        top5 = np.partition(-prediction[0], 5)
        top5_result = -top5[:5]
        #print(top5_result)

        index=random.choices(top5_args,top5_result)
        #print(type(index))
        #index = random.choices(np.arange(len(int_to_note)),prediction[0])
        print(int_to_note[index[0]])
        result = int_to_note[index[0]]
        prediction_outputA.append(result)

        pattern.append(index[0])
        pattern = pattern[1:len(pattern)]
        
    for note_index in range(25):
        prediction_input = np.reshape(pattern, (1, len(pattern), 1))
        prediction_input = prediction_input / float(n_vocab)

        
        ### Copy the line below from your above implementation.
        prediction = model.predict(prediction_input)
        
        top5arg = np.argpartition(-prediction[0], 15)
        top5_args = top5arg[:15]
        #print(top5_args)
        
        top5 = np.partition(-prediction[0], 15)
        top5_result = -top5[:15]
        #print(top5_result)

        index=random.choices(top5_args,top5_result)
        #print(type(index))
        #index = random.choices(np.arange(len(int_to_note)),prediction[0])
        print(int_to_note[index[0]])
        result = int_to_note[index[0]]
        prediction_outputB.append(result)

        pattern.append(index[0])
        pattern = pattern[1:len(pattern)]
    
    prediction_output=prediction_outputA+prediction_outputB+prediction_outputA
    
    return prediction_output